In [36]:
import os

In [37]:
import random

In [38]:
from os.path import join

In [39]:
data_prefix = "/project/projectdirs/dasrepo/gordon_bell/deep_learning/data/climate/CAM5_0.25"

In [40]:
source_file_prefix = "/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/"

In [41]:
names = ["2xCO2", "SST_plus2",  "climo"]

In [42]:
im_txt_file = join(source_file_prefix,"image_files.txt")
lbl_txt_file = join(source_file_prefix,"label_files.txt")
crop_ind_txt_file = join(source_file_prefix,"crop_indices.txt")
for name in names:
    path = join(data_prefix,name)
    imdir = join(path,"images")
    ims = os.listdir(imdir)
    lbldir = join(path,"labels")
    lbls = os.listdir(lbldir)
    ims.sort()
    lbls.sort()
    for crop_ind in range(12):
        with open(im_txt_file, "a") as f:
            for impath in ims:
                f.write(join(imdir,impath) + "\n")
        with open(lbl_txt_file, "a") as f:
            for lblpath in lbls:
                f.write(join(lbldir, lblpath) + "\n")
        with open(crop_ind_txt_file, "a") as f:
            for _ in lbls:
                f.write(str(crop_ind) + "\n")

In [45]:
#shuffle everything the same
for txt_file in [im_txt_file, lbl_txt_file, crop_ind_txt_file]:
    with open(txt_file, "r") as f:
        random.seed(3)
        lines = f.readlines()
        random.shuffle(lines)
    with open(txt_file, "w") as f:
        f.writelines(lines)
    
        

        
#         with open(join(source_file_prefix,"label_files.txt"), "a") as f:
#     for lblpath in lbls:
#         f.write(lblpath)
# with open(join(source_file_prefix,"crop_indices.txt"), "a") as f:
            
    

In [38]:
import random

In [39]:
a = range(5)

In [40]:
b = range(5)

In [41]:
b

[0, 1, 2, 3, 4]

In [42]:
a

[0, 1, 2, 3, 4]

In [43]:
random.seed(3)

In [44]:
random.shuffle(a)

In [45]:
a

[0, 3, 4, 2, 1]

In [46]:
random.seed(3)

In [47]:
random.shuffle(b)

In [48]:
b

[0, 3, 4, 2, 1]